In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import time
from datetime import datetime
import scipy.spatial
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
dtypes = pd.Series({'idaviso': np.dtype('uint32'), 'idpostulante': np.dtype('object')})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

postulaciones = pd.read_csv('Data/Postulaciones/fiuba_4_postulaciones-reduced.csv',dtype=column_types)
postulaciones = postulaciones.drop(columns=['Unnamed: 0'])

In [3]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3401623 entries, 0 to 3401622
Data columns (total 3 columns):
idaviso             uint32
idpostulante        object
fechapostulacion    object
dtypes: object(2), uint32(1)
memory usage: 64.9+ MB


In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
data = pd.DataFrame()
data['counts'] = postulaciones["idpostulante"].value_counts()
data['log_counts'] = np.log(data["counts"]+1)
g = data["log_counts"].plot.hist(bins=100, color="lightgreen")
g.set_title("Histograma de postulaciones por postulante (log) [Postulaciones a n avisos]", fontsize=18)
g.set_xlabel("Postulaciones (log)",fontsize=18)
g.set_ylabel("Frecuencia", fontsize=18)

Text(0,0.5,'Frecuencia')

# Busco intereses de postulacion por postulante

In [5]:
unique_postulantes = postulaciones.copy()
unique_postulantes.drop_duplicates(subset='idpostulante',inplace=True)
unique_postulantes.drop(columns=['fechapostulacion','idaviso'],inplace=True)
unique_postulantes['nombre_zona_interes'] = ""
unique_postulantes['nivel_laboral_interes'] = ""
unique_postulantes['tipo_de_trabajo_interes'] = ""
unique_postulantes['nombre_area_interes'] = ""
unique_postulantes.set_index('idpostulante',inplace=True)
unique_postulantes.head()

,nombre_zona_interes,nivel_laboral_interes,tipo_de_trabajo_interes,nombre_area_interes
idpostulante,,,,
NM5M,,,,
5awk,,,,
ZaO5,,,,
NdJl,,,,
eo2p,,,,


In [69]:
unique_postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200888 entries, NM5M to 2zP6J9V
Data columns (total 4 columns):
nombre_zona_interes        200888 non-null object
nivel_laboral_interes      200888 non-null object
tipo_de_trabajo_interes    200888 non-null object
nombre_area_interes        200888 non-null object
dtypes: object(4)
memory usage: 12.7+ MB


In [7]:
dtypes = pd.Series({'idaviso': np.dtype('uint32'), 'nombre_area_code': np.dtype('uint8'),
                   'denominacion_empresa_code': np.dtype('uint16'), 'nivel_laboral_code': np.dtype('uint8'),
                   'tipo_de_trabajo_code': np.dtype('uint8'), 'nombre_zona_code': np.dtype('uint8')})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))
optimized_avisos_detalle = pd.read_csv('Data/optimized_avisos_detalle.csv',dtype=column_types)
optimized_avisos_detalle.drop(columns=['Unnamed: 0'],inplace=True)

In [8]:
optimized_avisos_detalle.head()

,idaviso,nombre_area_code,denominacion_empresa_code,nivel_laboral_code,tipo_de_trabajo_code,nombre_zona_code
0,1111556097,51,3680,4,1,7
1,1111949392,90,3427,4,1,7
2,1112145935,86,2455,4,1,7
3,1112146010,145,866,4,1,7
4,1112211475,30,3230,4,1,7


In [28]:
postulaciones_sort_by_aviso = postulaciones.sort_values(by=['idaviso']).copy()
postulaciones_sort_by_aviso.drop(columns=['fechapostulacion'],inplace=True)
postulaciones_sort_by_aviso.drop_duplicates(subset=['idaviso','idpostulante'],inplace=True)
postulaciones_sort_by_aviso.set_index('idaviso',inplace=True)

In [56]:
unique_postulantes.loc['dYjbR9P']

nombre_zona_interes        
nivel_laboral_interes      
tipo_de_trabajo_interes    
nombre_area_interes        
Name: dYjbR9P, dtype: object

In [65]:
for aviso_detalle in list(zip(optimized_avisos_detalle['idaviso'], 
                              optimized_avisos_detalle['nombre_area_code'],
                             optimized_avisos_detalle['nivel_laboral_code'],
                             optimized_avisos_detalle['tipo_de_trabajo_code'],
                             optimized_avisos_detalle['nombre_zona_code'])):
    idaviso = aviso_detalle[0]
    try:
        df_aviso = postulaciones_sort_by_aviso.loc[idaviso]
        for postulante in list(zip(df_aviso['idpostulante'])):
            if str(aviso_detalle[4]) not in unique_postulantes['nombre_zona_interes'].loc[postulante[0]]:
                unique_postulantes['nombre_zona_interes'].loc[postulante[0]] += "-" + str(aviso_detalle[4])
            if str(aviso_detalle[2]) not in unique_postulantes['nivel_laboral_interes'].loc[postulante[0]]:
                unique_postulantes['nivel_laboral_interes'].loc[postulante[0]] +=  "-" + str(aviso_detalle[2])
            if str(aviso_detalle[3]) not in unique_postulantes['tipo_de_trabajo_interes'].loc[postulante[0]]:
                unique_postulantes['tipo_de_trabajo_interes'].loc[postulante[0]] += "-" + str(aviso_detalle[3])
            if str(aviso_detalle[1]) not in unique_postulantes['nombre_area_interes'].loc[postulante[0]]:
                unique_postulantes['nombre_area_interes'].loc[postulante[0]] += "-" + str(aviso_detalle[1])
    except KeyError:
        continue
    except AttributeError:
        if str(aviso_detalle[4]) not in unique_postulantes['nombre_zona_interes'].loc[postulante[0]]:
            unique_postulantes['nombre_zona_interes'].loc[postulante[0]] += "-" + str(aviso_detalle[4])
        if str(aviso_detalle[2]) not in unique_postulantes['nivel_laboral_interes'].loc[postulante[0]]:
            unique_postulantes['nivel_laboral_interes'].loc[postulante[0]] +=  "-" + str(aviso_detalle[2])
        if str(aviso_detalle[3]) not in unique_postulantes['tipo_de_trabajo_interes'].loc[postulante[0]]:
            unique_postulantes['tipo_de_trabajo_interes'].loc[postulante[0]] += "-" + str(aviso_detalle[3])
        if str(aviso_detalle[1]) not in unique_postulantes['nombre_area_interes'].loc[postulante[0]]:
            unique_postulantes['nombre_area_interes'].loc[postulante[0]] += "-" + str(aviso_detalle[1])

In [70]:
unique_postulantes

,nombre_zona_interes,nivel_laboral_interes,tipo_de_trabajo_interes,nombre_area_interes
idpostulante,,,,
NM5M,-7,-4-3,-1-2,-4-3
5awk,-7-1,-1-4,-1,-1-4
ZaO5,-7-1,-4-2-3,-1-2,-4-2-3
NdJl,-7-1,-0-1-4,-1,-0-1-4
eo2p,-7,-0-4,-1,-0-4
Ez8J,-7-1,-4-2,-1,-4-2
aOQq,-7-1,-4-2,-1,-4-2
8BkL,-7-1,-2-4-1-3,-1,-2-4-1-3
1d2B,-7,-4,-1,-4


In [77]:
unique_postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200888 entries, NM5M to 2zP6J9V
Data columns (total 4 columns):
nombre_zona_interes        200888 non-null object
nivel_laboral_interes      200888 non-null object
tipo_de_trabajo_interes    200888 non-null object
nombre_area_interes        200888 non-null object
dtypes: object(4)
memory usage: 12.7+ MB


In [72]:
unique_postulantes.to_csv('Data/unique_postulantes_con_intereses_postulaciones.csv')